# Загрузка Pandas и очистка данных

In [12]:
# стандартные библиотеки
import re
import json
from ast import literal_eval
from datetime import datetime
import math

import pandas as pd
import numpy as np

from datetime import datetime
import itertools

In [4]:
# загружаем датасет, содержащий
afinn = dict([line.replace('\n','').split('\t') for line in open("AFINN-111.txt")])
afinn


{'abandon': '-2',
 'abandoned': '-2',
 'abandons': '-2',
 'abducted': '-2',
 'abduction': '-2',
 'abductions': '-2',
 'abhor': '-3',
 'abhorred': '-3',
 'abhorrent': '-3',
 'abhors': '-3',
 'abilities': '2',
 'ability': '2',
 'aboard': '1',
 'absentee': '-1',
 'absentees': '-1',
 'absolve': '2',
 'absolved': '2',
 'absolves': '2',
 'absolving': '2',
 'absorbed': '1',
 'abuse': '-3',
 'abused': '-3',
 'abuses': '-3',
 'abusive': '-3',
 'accept': '1',
 'accepted': '1',
 'accepting': '1',
 'accepts': '1',
 'accident': '-2',
 'accidental': '-2',
 'accidentally': '-2',
 'accidents': '-2',
 'accomplish': '2',
 'accomplished': '2',
 'accomplishes': '2',
 'accusation': '-2',
 'accusations': '-2',
 'accuse': '-2',
 'accused': '-2',
 'accuses': '-2',
 'accusing': '-2',
 'ache': '-2',
 'achievable': '1',
 'aching': '-2',
 'acquit': '2',
 'acquits': '2',
 'acquitted': '2',
 'acquitting': '2',
 'acrimonious': '-3',
 'active': '1',
 'adequate': '1',
 'admire': '3',
 'admired': '3',
 'admires': '3',


In [6]:
df = pd.read_csv('main_task.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [1]:
# Заполняем пропуски в столбце 'Number of Reviews' значением 0
def extract_dates_from_reviews(reviews_series):
    """Извлекает даты из столбца Reviews"""

    def get_date(reviews, i):
        return reviews[1][i] if len(reviews[1]) > i else None

    date1 = reviews_series.apply(lambda x: get_date(x, 0)).astype(np.datetime64)
    date2 = reviews_series.apply(lambda x: get_date(x, 1)).astype(np.datetime64)

    return date1, date2
#date1, date2 = extract_dates_from_reviews(df.Reviews)

In [7]:
def get_lowercase_words(s):
    """Функция оставляет только буквы и переводит их в строчные"""
    return [m[0].lower() for w in s.split(" ") for m in [re.findall(r"([A-Za-z0-9_]+)", w)] if m]

In [8]:
# Функция для анализа тональности текста (sentiment analysis, text mining)
def get_sentiment_analysis(words):
    return np.sum([int(afinn.get(word)) for word in words if afinn.get(word) is not None])

In [22]:
def string_to_list(string):
    # Функция преобразования строкового представления списка в тип list
    if type(string) != str or (not string.startswith('[') and not string.endswith(']')):
        return ['']
    return literal_eval(string.replace('nan', '""'))

In [13]:
# также добавим признак, обозначающий общее настроение отзывов
df['Reviews'].dropna(inplace=True)
df['Reviews'] = df['Reviews'].apply(string_to_list)
reviews_text_series = df["Reviews"].apply(lambda x: " ".join(x[0]))
words_series = reviews_text_series.apply(get_lowercase_words)
#df_output["Reviews Evaluated"] = words_series.apply(get_value_of_words)
words_series


0        [good, food, at, your, doorstep, a, good, hote...
1             [unique, cuisine, delicious, nepalese, food]
2             [catch, up, with, friends, not, exceptional]
3                                                       []
4        [best, place, to, try, a, bavarian, food, nice...
                               ...                        
39995    [the, real, italian, experience, wonderful, fo...
39996    [parisian, atmosphere, bit, pricey, but, inter...
39997    [good, by, swedish, standards, a, hidden, jewe...
39998    [underground, restaurant, oldest, restaurant, ...
39999                        [average, nice, an, informal]
Name: Reviews, Length: 40000, dtype: object

In [14]:
# Функция для нормализации столбца
def normalize(col):
    center = col - col.mean()
    norm = np.sqrt((col**2).sum())
    return center/norm


In [15]:
cuisines_number_by_city = df.explode("Cuisine Style").groupby('City')[
        'Cuisine Style']
cuisines_number_by_city.aggregate(lambda x: len(set(x.values)))

City
Amsterdam      588
Athens         253
Barcelona      861
Berlin         656
Bratislava     136
Brussels       418
Budapest       426
Copenhagen     315
Dublin         370
Edinburgh      350
Geneva         228
Hamburg        293
Helsinki       193
Krakow         235
Lisbon         426
Ljubljana      121
London        1876
Luxembourg     118
Lyon           218
Madrid         732
Milan          598
Munich         354
Oporto         208
Oslo           243
Paris          976
Prague         541
Rome           727
Stockholm      329
Vienna         483
Warsaw         317
Zurich         293
Name: Cuisine Style, dtype: int64

In [17]:
# \w	[0-9a-z_] любая цифра, буква или знак подчеркивания
# + - одно или болле вхождений
# * - ноль или болле вхождений
# \s [ \t\v\r\n\f] пробельный символ
#Так как датасает о ресторанах в Европе, то пропуски в столбце 'Cuisine Style' заполняем значением 'European'
df['Cuisine Style'].fillna('European', inplace=True)

# Преобразеум строку в список
if(isinstance(df['Cuisine Style'][0], str)):
    df['Cuisine Style'] = df['Cuisine Style'].str.findall('\w+\s*\w*\s*\w*')

In [18]:
# создаём пустое множество для хранения уникальных значений кухонь
cuisines = set()  
for styles in df['Cuisine Style']:
    for item in styles:
        cuisines.add(item)


def find_item(cell):
    if item in cell:
        return 1
    return 0

# добвляем столбцы
for item in cuisines:
    df[item] = df['Cuisine Style'].apply(find_item)

In [21]:
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer


def get_dummy_values(column, prefix):
    # Функция получения dummy столбцов из столбца со списком
    preprocessing = MultiLabelBinarizer()
    dummy_values = preprocessing.fit_transform(column)
    print(preprocessing.classes_)
    return pd.DataFrame(dummy_values, columns=' ' + preprocessing.classes_)


if(type(df['Cuisine Style'][0]) == str):
    df['Cuisine Style'] = df['Cuisine Style'].apply(string_to_list)

map_price_range = {
    '$': 1,
    '$$ - $$$': 2,
    '$$$$': 3
}

In [23]:
#pat = re.compile('\w+\s*\w*\s*\w*')
#cuisine_styles = df['Cuisine Style'].dropna().apply(lambda x: ['default'] if pd.isnull(x) else pat.findall(x))
# cuisine_styles.explode().value_counts(ascending=False)
#
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
#df = pd.get_dummies(df, columns=['City', 'Price Range'], dummy_na=True)
corr = df.corr(method='spearman')
#list(corr.Rating[abs(corr.Rating)>=0.05].index)
df.groupby(
        'City')['Ranking'].rank(pct=True)

0        0.408005
1        0.602439
2        0.021886
3        0.529002
4        0.236282
           ...   
39995    0.082044
39996    0.464162
39997    0.651220
39998    0.302613
39999    0.774965
Name: Ranking, Length: 40000, dtype: float64

In [29]:
def feature_engineering(df_output):
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(
        string_to_list)
    df_output['Reviews'] = df_output['Reviews'].apply(string_to_list)

    # Замена условных обозначений 'Price Range' на числа от 0 до 2
    #df_output['Price Range'].replace(map_price_range, inplace=True)

    # Feature Engineering для Cuisine Style
    cuisines = set()  # создаём пустое множество для хранения уникальных значений кухонь
    for cuisinesList in df_output['Cuisine Style']:
        for cuisine in cuisinesList:
            cuisines.add(cuisine)

    # Добавляем столбцы для каждого типа кухни
    for item in cuisines:
        df_output[item] = df_output['Cuisine Style'].apply(find_item)

    # Добавляем признак - количество кухонь
    df_output['Cuisine Styles Count'] = df_output['Cuisine Style'].apply(len)

    reviews_first, reviews_second = extract_dates_from_reviews(df_output['Reviews'])

    between_reviews_delta = (reviews_second - reviews_first).dt.days
    df_output['Between Reviews delta is NaN'] = between_reviews_delta.isna().astype(
        'uint8')
    between_reviews_delta = between_reviews_delta.fillna(0)
    df_output['Between Reviews delta'] = between_reviews_delta
    
    # нормализация столбца Ranking по городу
    df_output['Ranking Normal'] = df_output.groupby(
        'City')['Ranking'].rank(pct=True)

    # среднее количество отзывов по городам
    reviews_number_by_city = df_output.groupby(
        'City')['Number of Reviews'].mean()
    df_output['Mean number of Reviews by city'] = df_output["City"].apply(
        lambda city: reviews_number_by_city[city])

    # Сделаем подсчет количества кухонь по городам
    cuisines_number_by_city = df_output.explode("Cuisine Style").groupby('City')[
        'Cuisine Style'].aggregate(lambda x: len(set(x.values)))
    
    df_output['Cuisines styles number by city'] = df_output["City"].apply(
        lambda city: cuisines_number_by_city[city])

    # также добавим признак, обозначающий общее настроение отзывов
    reviews_text_series = df_output["Reviews"].apply(lambda x: " ".join(x[0]))
    words_series = reviews_text_series.apply(get_lowercase_words)
    df_output["Sentiment Analysis of Reviews"] = words_series.apply(
        get_sentiment_analysis)

    # создаем список столюцов с количественными признаками
    cols = ["Number of Reviews", "Mean number of Reviews by city", "Between Reviews delta",
            "Sentiment Analysis of Reviews", "Cuisine Styles Count", "Cuisines styles number by city"]

    for col in cols:
        df_output[col] = normalize(df_output[col])

In [26]:
def preprocessing(df_output):
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id', 'ID_TA'], axis=1, inplace=True)

In [27]:
 # функция для заполнения пропусков
def fill_NAN(df_output):
    df_output['Cuisine Style'].fillna('European', inplace=True)

    df_output["Number of Reviews is NaN"] = pd.isna(
        df_output["Number of Reviews"]).astype('uint8')
    df_output['Number of Reviews'].fillna(0, inplace=True)

    df_output['Reviews is NaN'] = pd.isna(df_output['Reviews']).astype('uint8')
    df_output['Reviews'].fillna('[[],[]]', inplace=True)

    df_output['Price Range is NaN'] = pd.isna(
        df_output['Price Range']).astype('uint8')
    df_output['Price Range'].fillna(
        df_output['Price Range'].mode()[0], inplace=True)

In [31]:
def encoding(df_output):
    return pd.get_dummies(df_output, columns=['City', 'Price Range'], dummy_na=True)

In [32]:
def clean_model(df_output):
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [
        s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis=1, inplace=True)

In [24]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''

    df_output = df_input.copy()

    # ################### 1. Предобработка ##############################################################
    preprocessing(df_output)

    # ################### 2. NAN ##############################################################
    fill_NAN(df_output)
    
    # ################### 3. Feature Engineering ####################################################
    feature_engineering(df_output)
    
    # ################### 4. Encoding ##############################################################
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    encoding(df_output)
    
    # ################### 5. Clean ####################################################
    clean_model(df_output)

    return df_output

In [33]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

df_preproc = preproc_data(pd.read_csv('main_task.csv'))

In [34]:
corr = df_preproc.corr(method='spearman')
# Удалим столбце с корреляцией менее 0.01
#df_preproc.drop(list(corr.Rating[abs(corr.Rating)<0.001].index), axis = 1, inplace=True)

In [36]:
corr

,Ranking,Rating,Number of Reviews,Number of Reviews is NaN,Reviews is NaN,Price Range is NaN,,Mongolian,South American,Halal,...,Pizza,"Yun""""",Austrian,Cuisine Styles Count,Between Reviews delta is NaN,Between Reviews delta,Ranking Normal,Mean number of Reviews by city,Cuisines styles number by city,Sentiment Analysis of Reviews
Ranking,1.000000,-0.341748,-0.413602,0.114939,NaN,0.290341,-0.065616,-0.065616,-0.065616,-0.065616,...,-0.065616,-0.065616,-0.065616,-0.410076,0.257786,0.065913,0.646977,0.268564,0.661421,-0.263182
Rating,-0.341748,1.000000,-0.076750,0.063881,NaN,-0.020953,0.006831,0.006831,0.006831,0.006831,...,0.006831,0.006831,0.006831,0.076335,0.034467,0.044270,-0.516386,0.010061,-0.046771,0.186909
Number of Reviews,-0.413602,-0.076750,1.000000,-0.422733,NaN,-0.654394,0.023909,0.023909,0.023909,0.023909,...,0.023909,0.023909,0.023909,0.653212,-0.600486,-0.251673,-0.650291,0.188102,0.027259,0.356386
Number of Reviews is NaN,0.114939,0.063881,-0.422733,1.000000,NaN,0.266923,-0.007561,-0.007561,-0.007561,-0.007561,...,-0.007561,-0.007561,-0.007561,-0.195142,0.422352,0.280803,0.206480,-0.049119,-0.007880,-0.215495
Reviews is NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Between Reviews delta,0.065913,0.044270,-0.251673,0.280803,NaN,0.232609,-0.016356,-0.016356,-0.016356,-0.016356,...,-0.016356,-0.016356,-0.016356,-0.112713,0.664854,1.000000,0.112740,-0.001469,-0.026876,-0.312231
Ranking Normal,0.646977,-0.516386,-0.650291,0.206480,NaN,0.462136,-0.035144,-0.035144,-0.035144,-0.035144,...,-0.035144,-0.035144,-0.035144,-0.541179,0.388608,0.112740,1.000000,-0.000479,-0.001002,-0.400126
Mean number of Reviews by city,0.268564,0.010061,0.188102,-0.049119,NaN,-0.121237,-0.097304,-0.097304,-0.097304,-0.097304,...,-0.097304,-0.097304,-0.097304,0.141936,-0.097485,-0.001469,-0.000479,1.000000,0.228579,0.044604
Cuisines styles number by city,0.661421,-0.046771,0.027259,-0.007880,NaN,-0.024713,0.003946,0.003946,0.003946,0.003946,...,0.003946,0.003946,0.003946,-0.018751,-0.061992,-0.026876,-0.001002,0.228579,1.000000,0.012678


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [35]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df_preproc.drop(['Rating'], axis = 1)
y = df_preproc['Rating']

# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
#df_preproc.corr()

# Создаём, обучаем и тестируем модель

In [37]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics  # инструменты для оценки точности модели
# Создаём модель
regr = RandomForestRegressor(
    n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

# округляем результат, чтобы он был кратный 0.5, т.к. возможные значения рейтинга кратны 0.5
y_pred = np.round(y_pred*2) / 2
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   13.8s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s


MAE: 0.1706875


[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
